In [2]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

/home/drlecter/.pyenv/versions/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [5]:
dataset = load_dataset("chirunder/text_messages")

In [5]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [6]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [7]:
# Cargar el tokenizer del modelo GPT2 de Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Definir una función para tokenizar y ajustar la longitud a 300 tokens
def tokenize_text(text):
    try:
        tokens = tokenizer.encode(text, truncation=True, padding='max_length', max_length=300)
        return tokens
    except Exception as e:
        print(f"Error tokenizing text: {text}\n{e}")
        return [tokenizer.pad_token_id] * 300  # Return a padded token sequence in case of error

# Verificar el tamaño del DataFrame
print(f"DataFrame size: {df.shape}")

# Trabajar con una muestra si el DataFrame es muy grande
df_sample = df.sample(n=1000, random_state=42) if len(df) > 1000 else df

# Aplicar la tokenización a la columna 'texto' del DataFrame
df_sample['tokenized_text'] = df_sample['texto'].apply(tokenize_text)

# Crear el nuevo DataFrame con las columnas requeridas
df_tokenized = df_sample[['tokenized_text', 'texto', 'n']].copy()
df_tokenized.columns = ['vector', 'texto', 'n']

print(df_tokenized.columns)


DataFrame size: (11615290, 2)
Index(['vector', 'texto', 'n'], dtype='object')


## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [8]:
# Resuelve Conectar a una base de datos local de LanceDB
database = lancedb.connect("./.lancedb")

# Crear una tabla en la base de datos con el DataFrame tokenizado
database.create_table("tabla_tokenizada", df_tokenized)

# Mostrar las primeras filas de la tabla creada para verificar
print(database["tabla_tokenizada"].head()) 

pyarrow.Table
vector: fixed_size_list<item: float>[300]
  child 0, item: float
texto: string
n: int64
----
vector: [[[56,929,11,326,338,...,50256,50256,50256,50256,50256],[40,4030,616,5852,37976,...,50256,50256,50256,50256,50256],[21352,1660,318,635,1049,...,50256,50256,50256,50256,50256],[1026,318,407,3306,284,...,50256,50256,50256,50256,50256],[9690,287,5963,355,314,...,50256,50256,50256,50256,50256]]]
texto: [["Yup, that's another, and there are several others that are really the same as far as I can tell.","I kept my seatbelt on.","Hot water is also great to have for hypothermia, emergency medicine sterilization, etc.","It is not necessary to slather the bolt in it.","Thanks in advance as I'm sure I'll receive an answer or be directed to another post from the past."]]
n: [[19,5,13,10,19]]


## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [9]:
# Resuelve el task 3 aqui# Realizar una consulta a la tabla en la base de datos
result_df = (
    database["tabla_tokenizada"].search()
    .where("n <= 5")  # Condición: seleccionar filas donde "n" es menor o igual a 5
    .select(["texto", "vector"])  # Seleccionar las columnas "texto" y "vector"
    .limit(10)  # Limitar los resultados a 11 filas
    .to_pandas()  # Convertir el resultado a un DataFrame de pandas
)

# Mostrar los resultados
print(result_df)

                             texto  \
0           I kept my seatbelt on.   
1  How about getting a girlfriend.   
2        I'm wheeling it tomorrow.   
3                    thanks frick.   
4                       Andrew No.   
5       I can't believe its gonna.   
6                Even on Hardcore.   
7                    Yah found it.   
8                  Water is Clear.   
9        Yeah, I wasnt that stuck.   

                                              vector  
0  [40.0, 4030.0, 616.0, 5852.0, 37976.0, 319.0, ...  
1  [2437.0, 546.0, 1972.0, 257.0, 11077.0, 13.0, ...  
2  [40.0, 1101.0, 7825.0, 278.0, 340.0, 9439.0, 1...  
3  [27547.0, 1216.0, 624.0, 13.0, 50256.0, 50256....  
4  [20508.0, 1400.0, 13.0, 50256.0, 50256.0, 5025...  
5  [40.0, 460.0, 470.0, 1975.0, 663.0, 8066.0, 13...  
6  [6104.0, 319.0, 49691.0, 13.0, 50256.0, 50256....  
7  [56.0, 993.0, 1043.0, 340.0, 13.0, 50256.0, 50...  
8  [19184.0, 318.0, 11459.0, 13.0, 50256.0, 50256...  
9  [10995.0, 11.0, 314.0, 373.0

- Query en SQL equivalente:
- Explicacion: 

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [4]:
# Cargar el tokenizer del modelo GPT2 de Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Inventar un mensaje de texto
mensaje_amigo = "Hey, ¿qué planes tienes para el fin de semana? Me gustaría salir a caminar."

# Tokenizar el mensaje
mensaje_tokenizado = tokenizer.encode(mensaje_amigo, truncation=True, padding='max_length', max_length=300)

# Definir una función para tokenizar y ajustar la longitud a 300 tokens
def tokenize_text(text):
    tokens = tokenizer.encode(text, truncation=True, padding='max_length', max_length=300)
    return tokens

# Aplicar la tokenización a la columna 'texto' del DataFrame
df['tokenized_text'] = df['texto'].apply(tokenize_text)

# Crear el nuevo DataFrame con las columnas requeridas
df_tokenized = df[['tokenized_text', 'texto', 'n']].copy()
df_tokenized.columns = ['vector', 'texto', 'n']

# Conectar a una base de datos local de LanceDB
database = lancedb.connect("./.lancedb")

# Crear una tabla en la base de datos con el DataFrame tokenizado
database.create_table("tabla_tokenizada", df_tokenized)

# Mostrar las primeras filas de la tabla creada para verificar
print(database["tabla_tokenizada"].head())

NameError: name 'df' is not defined

In [ ]:
# Realizar una consulta a la tabla en la base de datos usando el vector del mensaje tokenizado
result = (
    database["tabla_tokenizada"].search()
    .where("n <= 5")  # Ejemplo de condición para la consulta
    .select(["texto", "vector"])  # Seleccionar las columnas "texto" y "vector"
    .limit(11)  # Limitar los resultados a 11 filas
    .to_pandas()  # Convertir el resultado a un DataFrame de pandas
)

# Mostrar los resultados
print(result)

In [ ]:
# Realizar una consulta vectorial
similar_result = (
    database["tabla_tokenizada"].search()
    .vector_search(mensaje_tokenizado)  # Realizar la búsqueda vectorial con el mensaje tokenizado
    .select(["texto"])  # Seleccionar solo la columna "texto"
    .limit(1)  # Limitar el resultado a la fila más parecida
    .to_pandas()  # Convertir el resultado a un DataFrame de pandas
)

# Decodificar el resultado
mensaje_mas_parecido = similar_result['texto'].iloc[0]

# Mostrar el mensaje más parecido
print(f"Mensaje más parecido: {mensaje_mas_parecido}")